In [1]:
import pandas as pd

In [2]:
file_path = 'SentiWord_Dict.txt'

In [3]:
# 사전 데이터 불러오기
df = pd.read_csv('SentiWord_Dict.txt', delimiter='\t', names=['단어', '점수'], header=None)

In [4]:
# 한글 부분만 필요하기 때문에, 사전에서 한글필터링 - 정규표현식
df = df[df['단어'].str.match(r'^[가-힣]+$')]

In [13]:
df.head(10)

,단어,점수
44,가까스로,0.0
46,가까이하다,1.0
47,가꾸러뜨리다,-1.0
48,가꾸러트리다,-1.0
49,가난,-2.0
50,가난뱅이,-2.0
51,가난살이,-2.0
52,가난살이하다,-2.0
53,가난설음,-2.0
54,가난에,-2.0


In [14]:
community_contents = pd.read_csv('community_token_final.csv')

C:\Users\hydrl\AppData\Local\Temp\ipykernel_2036\287724821.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  community_contents = pd.read_csv('community_token_final.csv')


In [15]:
# 데이터 타입 확인
print(community_contents.dtypes)

닉네임           object
날짜            object
내용            object
종목            object
내용_명사         object
내용_형용사        object
내용_부사         object
내용_동사         object
내용_명사_str     object
내용_형용사_str    object
내용_부사_str     object
내용_동사_str     object
dtype: object


In [5]:
community_contents = pd.read_csv('community_token_final.csv', low_memory=False)

In [17]:
# 데이터 타입 확인
print(community_contents.dtypes)

닉네임           object
날짜            object
내용            object
종목            object
내용_명사         object
내용_형용사        object
내용_부사         object
내용_동사         object
내용_명사_str     object
내용_형용사_str    object
내용_부사_str     object
내용_동사_str     object
dtype: object


In [21]:
community_contents.head(2)

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다"
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있"


df.head()

In [22]:
df.head()

,단어,점수
44,가까스로,0.0
46,가까이하다,1.0
47,가꾸러뜨리다,-1.0
48,가꾸러트리다,-1.0
49,가난,-2.0


In [23]:
def calculate_positive_score(contents):
  total_score = 0.0
  
  positive_dict = pd.read_csv('SentiWord_Dict.txt', delimiter='\t', names=['단어', '점수'], header=None)
  positive_dict = positive_dict[positive_dict['단어'].str.match(r'^[가-힣]+$')]
  
  positive_scores = []
  
  for nouns in contents['내용_명사']:
    total_score = 0.0

    for noun in nouns:  # nouns는 리스트
        score = positive_dict.loc[positive_dict['단어'] == noun, '점수']
        if not score.empty:
            total_score += score.values[0]  # 점수 합산
    positive_scores.append(total_score)
  
  
  # 긍정 점수에 따라 새로운 열 추가
  contents['긍정점수_명사'] = [1 if score > 0 else (0 if score == 0 else -1) for score in positive_scores]
  
  return contents

In [24]:
# 20분소요
result = calculate_positive_score(community_contents)
result.head(2)

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,긍정점수_명사
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다",-3.0
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있",0.0


In [6]:
result['긍정점수_명사'].value_counts()

NameError: name 'result' is not defined

In [23]:
# 사전에 없는 단어를 저장할 집합
missing_nouns = set()

In [24]:
existing_words = set(df['단어'])

In [25]:
# 모든 행을 돌면서 사전에 없는 단어 찾기
for nouns in community_contents['내용_명사']:
    for noun in nouns:  # 내부 리스트의 각 단어 순회
        clean_noun = noun.strip()  # 공백 제거
        if len(clean_noun) > 1 and clean_noun not in existing_words:  # 1글자 단어 제외
            missing_nouns.add(clean_noun)

In [26]:
missing_nouns

set()

In [51]:
# 특정 단어 확인
specific_word = '디자인'  # 확인하고자 하는 단어

if specific_word in existing_words:
    print(f"'{specific_word}'는 existing_words에 존재합니다.")
else:
    print(f"'{specific_word}'는 existing_words에 존재하지 않습니다.")

'디자인'는 existing_words에 존재하지 않습니다.


In [ ]:
existing_words = set(df['단어'])  # 중복 제거를 위해 set으로 변환

# 사전에 없는 단어를 저장할 집합
missing_nouns = set()

# 모든 행을 돌면서 사전에 없는 단어 찾기
for nouns in result['내용_명사']:
    for noun in nouns:
        if noun not in existing_words:
            missing_nouns.add(noun)